In [ ]:
from ipyleaflet import *
import ipywidgets
from ipywidgets.embed import embed_minimal_html
from ipywebrtc import WidgetStream, ImageRecorder

import json
import numpy as np
import matplotlib.pyplot as plt
#import gdal

import rasterio
from rasterio import plot
from rasterio._base import gdal_version
import contextily

import ee
from geemap import *

import osmnx as ox
# from OSMPythonTools.api import OSM_api

In [ ]:
# To run once for setting up earth engine
#ee.Authenticate()

In [ ]:
ee.Initialize()

# Test

## setup

In [ ]:
date_start = '2021-06-01'
date_end = '2021-07-15'

aoi = type('', (), {})
aoi.yMin = 12.0
aoi.xMin = 4.0
aoi.yMax = 14.0
aoi.xMax = 6.0

#aoiEE = ee.Geometry.Point([5.22, 13.05]).buffer(50000).bounds();

aoiEE = ee.Geometry.Rectangle([aoi.xMin, aoi.yMin, aoi.xMax, aoi.yMax])

## Fetch

In [ ]:
s2 = (ee.ImageCollection('COPERNICUS/S2_SR')
      .filterDate(date_start, date_end)
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
      .mosaic().clip(aoiEE)
     )
s2_vis = {
    "min": 0, 
    "max": 3000, 
    "bands": ['B4', 'B3', 'B2'],
}

l8 = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
      .filterDate(date_start, date_end)
      .filter(ee.Filter.lt('CLOUD_COVER', 20))
      .mosaic().clip(aoiEE)
     )
l8_vis = {
    'min': 0, 
    'max': 3000, 
    'bands': ['B4', 'B3', 'B2'],
}

nightNoaa = (ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')
             .filterDate(date_start, date_end)
             .mosaic().clip(aoiEE)
            )
nightNoaa_vis = {
    'min': -15.0, 
    'max': 1000.0, 
    'bands': ['avg_rad'], 
    "palette": ['000000','ffffff'],
}

nightNoaaAlt = (ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG')
             .filterDate(date_start, date_end)
             .mosaic().clip(aoiEE)
            )
nightNoaaAlt = {
    'min': 0.0, 
    'max': 10.0, 
    'bands': ['avg_rad'], 
    "palette": ['000000','ffffff'],
}

#nightNasa = basemap_to_tiles(basemaps.NASAGIBS.ViirsEarthAtNight2012, date_start)
#nightNasa_vis = {
#    "min": 0,
#    "max": 60,
#}

popGHSL = (ee.ImageCollection("JRC/GHSL/P2016/POP_GPW_GLOBE_V1")
           .filterDate('2014-01-01', '2016-01-01')
           .mosaic().clip(aoiEE)
          ).select('population_count')
popGHSL_vis = {
  "min": 0.0,
  "max": 20.0,
  "palette": ['000000', 'ffffff'],
}

#osm = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik, date_start)

In [ ]:
features = []
step = 1
for x in range(4, 6, step):
    for y in range(12, 14, step):
        features.append(ee.Feature(ee.Geometry.Rectangle(x, y, x+step, y+step)))
testFeatures = ee.FeatureCollection(features)

In [ ]:
def updateLayerFromSlider(layer, layerVis, name, mask, value):
    m.addLayer(layer.updateMask(mask.gt(value)), layerVis, name + str(value))

slider= ipywidgets.widgets.IntSlider(0, -15, 100)

button = widgets.Button(description="Test slider!")
output = widgets.Output()


def on_button_clicked(b):
    with output:
        updateLayerFromSlider(s2, s2_vis, 'sentinel-2 night radiance: ', nightNoaa.select('avg_rad'), slider.value)

button.on_click(on_button_clicked)

In [ ]:
m = geemap.Map(center=(0, 0), zoom=4)
#m.fit_bounds([(aoi.yMin, aoi.xMin), (aoi.yMax, aoi.xMax)])
m.add_control(ScaleControl(position="topright"))
m.add_control(LayersControl())


m.addLayer(s2, s2_vis, 'sentinel-2')
m.addLayer(l8, l8_vis, 'landsat-8')

m.addLayer(nightNoaa, nightNoaa_vis, 'night Noaa')
#m.addLayer(nightNoaaAlt, nightNoaaAlt_vis, 'night Noaa Alt')
#m.add_layer(nightNasa)
m.addLayer(popGHSL, popGHSL_vis, 'population GHSL')

#m.add_layer(osm)
#m.add_osm_from_bbox(13, 13.1, 5.0, 5.1, {'building': True}, 'osm', info_mode=None)

m.addLayer(testFeatures.style(color='ffffff', width=0.5,fillColor='ffffff00'), {}, 'test')

display(slider)
display(button, output)

## Save

In [ ]:
def saveImageFromGeeData(eeData, vis_params, folderName, fileName, aoi, zoom = 14):
    offset = 100
    step = 50
    count = 0
    
    x_min = int(aoi.xMin*offset)
    x_max = int(aoi.xMax*offset)
    y_min = int(aoi.yMin*offset)
    y_max = int(aoi.yMax*offset)

    url = eeData.getMapId(vis_params)["tile_fetcher"].url_format
    for x in range(x_min, x_max, step):
        for y in range(y_min, y_max, step):
            count += 1
            
            east = x/offset
            west = x_max/offset if (x+step > x_max) else (x+step)/offset
            north = y/offset
            south = y_max/offset if (y+step > y_max) else (y+step)/offset

            if (y > y_max): y = y_max
            print(count,'e:',east,'w:',west,'n:',north,'s:',south)
            _, _ = contextily.bounds2raster(aoi.xMin,aoi.yMin,aoi.xMax,aoi.yMax,ll=True,path='img/'+folderName+'/'+fileName+'_'+str(count),source=url, zoom=zoom)
            print('Done')

#def saveImageFromProvidersSrc(src, fileName, aoi, zoom = 14):
#    img, _ = contextily.bounds2raster(aoi.xMin,aoi.yMin,aoi.xMax,aoi.yMax,ll=True,path=fileName,source=src, zoom=zoom)
#    return img

In [ ]:
# Test !!

offset = 100
step = 50
count = 0
x_min = int(3.35*offset)
x_max = int(4.7*offset)
y_min = int(11.3*offset)
y_max = int(12.5*offset)

for x in range(x_min, x_max, step):
    for y in range(y_min, y_max, step):
        count += 1

        east = x/offset
        west = x_max/offset if (x+step > x_max) else (x+step)/offset
        north = y/offset
        south = y_max/offset if (y+step > y_max) else (y+step)/offset

        print('Test'+'/'+'Sokoto'+' '+str(count),east,west,north,south)

In [ ]:
sentinelImg = saveImageFromGeeData(s2, s2_vis,"sokoto_s2", aoi, zoom=14)
landsatImg = saveImageFromGeeData(l8, l8_vis,"sokoto_l8", aoi, zoom=13)

noaaImg = saveImageFromGeeData(nightNoaa, nightNoaa_vis, "sokoto_noaa", aoi, zoom=8)

# NASAGIBS limit the zoom level to 8 max.
#blackMarbleImg = saveImageFromProvidersSrc(contextily.providers.NASAGIBS.ViirsEarthAtNight2012, "sokoto_night.tif", aoi, zoom=8)

popImg = saveImageFromGeeData(popGHSL, popGHSL_vis,"sokoto_ghsl", aoi, zoom=10)
#osmImg = saveImageFromProvidersSrc(contextily.providers.OpenStreetMap.Mapnik, "sokoto_osm.tif", aoi, zoom=10)


In [ ]:
# FAILED # plt.axis([aoi.yMin, aoi.yMax,aoi.xMin, aoi.xMax])

plt.title("Sentinel2")
with rasterio.open("sokoto_s2.tif") as r: 
    axis = plot.show(r).axis()

#plt.axis(axis)
#plt.title("Landsat 8")
#with rasterio.open("sokoto_l8.tif") as r: plot.show(r)

#plt.axis(axis)
#plt.title("VIIRS Nighttime")
#with rasterio.open("sokoto_noaa.tif") as r: plot.show(r)

#plt.axis(axis)
#plt.title("NASA Nighttime")
#with rasterio.open("sokoto_night.tif") as r: plot.show(r)

#plt.axis(axis)
#plt.title("Human settlement")
#with rasterio.open("sokoto_ghsl.tif") as r: plot.show(r)

#plt.axis(axis)
#plt.title("OpenStreetMap")
#with rasterio.open("sokoto_osm.tif") as r: plot.show(r)

